# Following Bot

Before doing anything, delete your `api_key.txt` file in Jupyter and replace it (by uploading) with your leader's `api_key.txt` file. Then, copy your leader's `RANDOM_STRING` into the bottom of the code below and run the code to save your leader's Airtable URL just like you did in the previous notebook.

In [ ]:
############################################
###FOLLOWING NOTEBOOK INITIALIZATION CODE###
############################################

##IMPORTS##
from easygopigo3 import EasyGoPiGo3
from dateutil import parser
import numpy as np
import time
import json
import requests

# Initialize easy gpg
my_easy_robot = EasyGoPiGo3()
# Make sure that all sensors and output devices are uncofigured to start
my_easy_robot.reset_all()
my_easy_robot.reset_speed()

print('Robot Initialized.')

###FUNCTIONS###
# get the Airtable API Key from a local file
def get_key(filename):
    fin = open(filename)
    for line in fin:
        key = line.strip()
    return key

# drive using driving_data retrieved from Airtable. We've built this function for you already and we'll use it later.
def drive_from_data(all_driving_data):
    my_easy_robot.reset_encoders() #reset encoders before using them to drive
    (move_type,left_encoder,right_encoder)=all_driving_data[1]
    if left_encoder==0 or right_encoder==0:
        return
    if move_type==0:
        #Drive forward to encoder values
        my_easy_robot.forward()
        while not (my_easy_robot.read_encoders()[0]>=left_encoder or my_easy_robot.read_encoders()[1]>=right_encoder):
            #print('Encoder values are: ', my_easy_robot.read_encoders())
            continue
        my_easy_robot.stop()
    elif move_type==1:
        #Turn right
        my_easy_robot.spin_right()
        while not (my_easy_robot.read_encoders()[0]>=left_encoder or my_easy_robot.read_encoders()[1]<=right_encoder):
            #print('Encoder values are: ', my_easy_robot.read_encoders())
            continue
        my_easy_robot.stop()
    elif move_type==-1:
        #Turn left
        my_easy_robot.spin_left()
        while not (my_easy_robot.read_encoders()[0]<=left_encoder or my_easy_robot.read_encoders()[1]>=right_encoder):
            #print('Encoder values are: ', my_easy_robot.read_encoders())
            continue
        my_easy_robot.stop()

'''---------------------------------------------------------'''
'''Put your partner's random string in the location below, just like last time.'''

PARTNER_API_KEY=get_key('api_key.txt')
BASETABLENAME = 'driving_bot'
'''---------------------------------------------------------'''
RANDOM_STRING = #Put your leader's random string here 
'''---------------------------------------------------------'''

url = 'https://api.airtable.com/v0/'+RANDOM_STRING+'/'+BASETABLENAME+"?api_key="+PARTNER_API_KEY

print(url)
print('Type the url above into your browser and ensure it takes you to your leader\'s airtable data')

**Ok, let's get started.**

The following bot's job will be to read the data from the Airtable that the driving bot creates, and then mimic the movements that the driving bot made using that data. The driving bot will be logging three values in the Airtable, `move_type`, `left_encoder`, and `right_encoder`. 

- `move_type = 0` means that the robot drove forward
- `move_type = 1` means that the robot turned right
- `move_type = -1` means that the robot turned left

For each of these moves we will read the values of the encoders after the move, namely: `left_encoder` and `right_encoder`

## Getting values

Let's make a function called `get_values(url)` that takes a `url`returns a list of all the driving data in Airtable, comprised each of two values. 

1. The first value will be `date_seconds`. Don't worry about this part of the code, it's already coded and will be used to figure out in what order the entries were posted.
2. The second value  will be `driving_data`. This is a tuple of three values,  `move_type`, `left_encoder`, and `right_encoder`.  [(link to what a tuple is since we haven't covered it that much in Python)](https://www.w3schools.com/python/python_tuples.asp). 

Our `driving_data` tuple will be defined like this:

`driving_data = (move_type, left_encoder, right_encoder)` 

An example driving data could be: `driving_data = (0, 200, 200)` which means the robot drove forward for 200 turns of the encoder on both motors. 

So, in order to make this tuple, we need to retrieve the `move_type`, `left_encoder`, and `right_encoder` values from the Airtable JSON dictionary.

Notice that the `requests.get(url)` function and the `json.loads(response.text)['records']` function have already been coded for you. They will save the response from Airtable as the dictionary `airtable_data`.

A loop has already been built for you to loop through each record in the data, and a `field` variable has already been defined in the loop. To get the values you need, take a look at EDL2020_Intro_to_IOT, specifically at *Part 3: Processing the JSON*. You'll want to find a way to extract a value like `left_encoder` from a field dictionary. For more information on dictionaries in Python, [click here.](https://www.w3schools.com/python/python_dictionaries.asp)

Finish the code below to get `move_type`, `left_encoder`, and `right_encoder`  from each field and save `driving_data` as:

`driving_data = (move_type, left_encoder, right_encoder)` 

in the space provided.

In [ ]:
def get_values(url):
    
    all_driving_data=[]
    
    response=requests.get(url) #get the response from the server
    airtable_data=json.loads(response.text)['records'] #load it into a dict
    
    for record in airtable_data:
        ##Getting the time##
        date_str=record['createdTime']
        date_obj=parser.isoparse(date_str)
        date_seconds = date_obj.hour*60*60 + date_obj.minute*60 + date_obj.second
        
        #Get the field from the record
        field=record['fields']
        '''--------------------------------------------'''
        '''
        In the space below:
        -Get move_type, left_encoder, and right_encoder from the field dictionary
        using their respective keys. 
        -Save these three values into a tuple called driving_data
        '''
        move_type= #get move_type from field
        left_encoder= #get left_encoder from field
        right_encoder= #get right_encoder from field
        
        driving_data= #save this as a tuple including the three variables above
        
        '''--------------------------------------------'''
        
        all_driving_data.append([date_seconds,driving_data]) #add the driving data stuff to all driving data
        
    all_driving_data.sort() #sort by time posted

    return all_driving_data

print('Your get_values function has been saved.')

## Try it out

Try out your get values function in the code below, by calling it with `url` (already saved in the computer's memory as you ran it in the initialization script) and printing the result.

Take a look at your data: does it match what was in your partner's Airtable? If so, we're ready to start driving!

In [ ]:
#Test get_values here
print(get_values(url))

## Following the data

We've created a `drive_from_data` function for you, that uses the `get_values` function you already created to drive according to the data it gets from your partner's Airtable. You can scroll back to the top of the page to take a look at how the function works. 

Ask your leader to use their `MAP` variable to create a fun driving pattern, but don't let them tell you what it is! Then, run the code below to watch your robot mimic theirs!

In [ ]:
###FOLLOW AND DRIVE CODE###
all_driving_data=get_values(url) #get the data from airtable using your get_values function

for driving_data in all_driving_data: #loop through the whole airtable dataset
    drive_from_data(driving_data) #drive using the data!
    
my_easy_robot.stop() #stop just in case

In [ ]:
#Emergency Stop
my_easy_robot.stop()